In [8]:
import numpy as np
import psycopg2
import cv2
import csv
import PIL
import os
import random
from PIL import Image
import pickle
import math

In [9]:
def quaternion_to_euler_angle(w, x, y, z):
    ysqr = y * y
    
    t0 = +2.0 * (w * x + y * z)
    t1 = +1.0 - 2.0 * (x * x + ysqr)
    X = math.degrees(math.atan2(t0, t1))

    t2 = +2.0 * (w * y - z * x)
    t2 = +1.0 if t2 > +1.0 else t2
    t2 = -1.0 if t2 < -1.0 else t2
    Y = math.degrees(math.asin(t2))

    t3 = +2.0 * (w * z + x * y)
    t4 = +1.0 - 2.0 * (ysqr + z * z)
    Z = math.degrees(math.atan2(t3, t4))

    return X, Y, Z

In [10]:
tables = ['lazare_bottom_floor_trajectory', 'lazare_top_floor_trajectory', 'lazare_second_floor_trajectory']

In [18]:
for table in tables:
    conn_string = "host='localhost' dbname='indoor_position' user='postgres' password='tiancai' port='5432'"
    conn = psycopg2.connect(conn_string)
    cur = conn.cursor()
    query = '''select field_8, field_4, field_5, field_6, field_7 from gare_st_lazare.%s''' % table
    cur.execute(query)
    results = cur.fetchall()
    conn.commit()

    for item in results:
        time  = item[0]
        qua_x = float(item[1])
        qua_y = float(item[2])
        qua_z = float(item[3])
        qua_w = float(item[4])
        euler_x, euler_y, euler_z = quaternion_to_euler_angle(qua_w, qua_x, qua_y, qua_z)
#         print(euler_x, euler_y, euler_z)
        
        query = '''update gare_st_lazare.lazare_frame_list set orientation = %s where time_abs = '%s'; ''' % (euler_z, time)
        cur.execute(query)
        
    conn.commit()